In [1]:
# !pip list

In [1]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from snowflake.snowpark.functions import udf
%matplotlib inline
import datetime as dt
import numpy as np
import seaborn as sns

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler
import snowflake.connector
# to save the trained scaler class
import joblib

#Snowflake connection info
# from config import snowflake_conn_prop



snowflake_conn_prop = {
   "account": "se58322-fsesg",
   "user": "Muhammad Nauman",
   "password": "Muhammad23",
   "role": "DEVELOPER",
   "database": "DL_ESG_DEV",
   "schema": "CUSTOMER",
   "warehouse": "WH_ESG_SUSTAINABILITY",
}

from snowflake.snowpark import version
print(version.VERSION)


connection_paramter = snowflake.connector.connect(account = "se58322-fsesg",
                        user="Muhammad Nauman",
                        password="Muhammad23",
                        role="DEVELOPER",
                        warehouse="WH_ESG_SUSTAINABILITY",
                        ocsp_fail_open=False)

session = Session.builder.configs(snowflake_conn_prop).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

/Users/muhammadnauman/opt/anaconda3/envs/ESG_MVP_IBM/lib/python3.8/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


(1, 0, 0)
[Row(CURRENT_WAREHOUSE()='WH_ESG_SUSTAINABILITY', CURRENT_DATABASE()='DL_ESG_DEV', CURRENT_SCHEMA()='CUSTOMER')]


In [2]:


qry = "SELECT * FROM DL_ESG_DEV.CUSTOMER.CUSTOMER"
qry = 'SELECT * FROM DL_ESG_DEV.ESG.TRIAL_SCO_ESG_262'

df = pd.read_sql(qry, connection_paramter)

/var/folders/nr/vyfq21gx5pn1p0025dclx97c0000gn/T/ipykernel_71744/1624574879.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qry, connection_paramter)


In [5]:
df

,date,asset_id,name,ticker,exchange_iso,exchange_mic,dom_country_iso,dom_region,exch_country_iso,exch_region,economic_sector,industry,esg,esg_e,esg_s,esg_g
0,2018-01-01,196,A. O. Smith Corporation,AOS,NYS,XNYS,USA,North America,USA,North America,Producer Manufacturing,Building Products,55.67,46.93,56.43,62.57
1,2018-01-01,208,Abbott Laboratories,ABT,NYS,XNYS,USA,North America,USA,North America,Health Technology,Medical Specialties,46.78,68.29,72.38,17.89
2,2018-01-01,209,"AbbVie, Inc.",ABBV,NYS,XNYS,USA,North America,USA,North America,Health Technology,Pharmaceuticals: Major,51.41,65.98,62.65,35.37
3,2018-01-01,270,Adobe Inc.,ADBE,NAS,XNAS,USA,North America,USA,North America,Technology Services,Packaged Software,64.47,68.40,67.56,61.60
4,2018-01-01,345,"Akamai Technologies, Inc.",AKAM,NAS,XNAS,USA,North America,USA,North America,Technology Services,Internet Software/Services,57.44,73.85,62.76,49.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,2018-01-01,94362,AT&T Inc.,T,SWX,XSWX,USA,North America,CHE,Europe,Communications,Major Telecommunications,53.34,63.41,51.20,49.93
491,2018-01-01,94615,Linde plc,LIN,ETR,XETR,GBR,Europe,DEU,Europe,Process Industries,Chemicals: Specialty,64.71,70.91,73.87,56.78
492,2018-01-01,94676,International Flavors & Fragrances Inc.,IFF,TAE,XTAE,USA,North America,ISR,Middle East,Consumer Non-Durables,Household/Personal Care,60.64,68.93,70.07,49.53
493,2018-01-01,94720,Exxon Mobil Corporation,XONA,ETR,XETR,USA,North America,DEU,Europe,Energy Minerals,Integrated Oil,55.49,58.74,54.11,54.64


In [9]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [10]:
articles_df = df.copy()
# articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

,date,asset_id,name,ticker,exchange_iso,exchange_mic,dom_country_iso,dom_region,exch_country_iso,exch_region,economic_sector,industry,esg,esg_e,esg_s,esg_g
0,2018-01-01,196,A. O. Smith Corporation,AOS,NYS,XNYS,USA,North America,USA,North America,Producer Manufacturing,Building Products,55.67,46.93,56.43,62.57
1,2018-01-01,208,Abbott Laboratories,ABT,NYS,XNYS,USA,North America,USA,North America,Health Technology,Medical Specialties,46.78,68.29,72.38,17.89
2,2018-01-01,209,"AbbVie, Inc.",ABBV,NYS,XNYS,USA,North America,USA,North America,Health Technology,Pharmaceuticals: Major,51.41,65.98,62.65,35.37
3,2018-01-01,270,Adobe Inc.,ADBE,NAS,XNAS,USA,North America,USA,North America,Technology Services,Packaged Software,64.47,68.40,67.56,61.60
4,2018-01-01,345,"Akamai Technologies, Inc.",AKAM,NAS,XNAS,USA,North America,USA,North America,Technology Services,Internet Software/Services,57.44,73.85,62.76,49.76


In [13]:
qry = 'SELECT * FROM DL_ESG_DEV.CUSTOMER.PORTFOLIO'

df_port = pd.read_sql(qry, connection_paramter)
# df

/var/folders/nr/vyfq21gx5pn1p0025dclx97c0000gn/T/ipykernel_71744/599122686.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_port = pd.read_sql(qry, connection_paramter)


In [25]:
interactions_df = df_port.copy()
interactions_df.shape
interactions_df['eventStrength'] = 1

In [26]:
users_interactions_count_df = interactions_df.groupby(['P_ID', 'R_ID']).size().groupby('P_ID').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 10].reset_index()[['P_ID']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 2000
# users with at least 5 interactions: 1987


In [27]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'P_ID',
               right_on = 'P_ID')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 40000
# of interactions from users with at least 5 interactions: 39890


In [55]:
## Count uniques
# print(interactions_full_df["eventStrength"].unique())
interactions_from_selected_users_df

,R_ID,P_ID,TICKER,BUY_DATE,BUY_VALUE,SHARES_,CURRENCY,OWNED,ADVISOR_ID,eventStrength
0,82ff1d93-d828-4d67-9b73-8f77ced7714a,00253276-5469-43fd-84a5-859a91e90086,BLCM,1633651200000,218,10,EUR,14.2666666667,0dad5b83-1334-4df6-83ff-1b83a3f703dd,1
1,a6f58d5a-fc14-4c66-b244-3b62688a9ad7,00253276-5469-43fd-84a5-859a91e90086,EMF,1449964800000,287,14,JPY,84.1,753946f3-4806-4cfa-9b1c-e329d1f19c90,1
2,2f5e2ea0-8185-4a94-aad7-f08eba574802,00253276-5469-43fd-84a5-859a91e90086,BITA,1521936000000,107,5,JPY,56.7,1767275a-5daf-4c22-b82f-3b309b98a2f2,1
3,03961a1b-3cd6-4179-b902-7f22340d53f0,00253276-5469-43fd-84a5-859a91e90086,FIDU,1612656000000,169,8,JPY,22.3,9452151e-d89b-4321-af7c-0dca5078459c,1
4,2362a56e-9a20-432a-87af-4a1194038117,00253276-5469-43fd-84a5-859a91e90086,BME,1408233600000,232,11,USD,99.9666666667,f82ea830-ea74-421a-b4cc-b7030e130ccb,1
...,...,...,...,...,...,...,...,...,...,...
39885,ffa6be99-f1a0-4e31-b39f-19dc7c93c32d,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,GJS,1531872000000,246,12,JPY,52.9333333333,f68c9f73-26d2-4d07-a1a4-a3b74eb454ac,1
39886,8e0594df-7e3e-4712-8315-ee600968cf73,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,NBEV,1383264000000,290,14,EUR,109.5,9b0f7277-91cc-4036-9db0-c12d58ba6a96,1
39887,c988a10c-fe29-4ce8-a1fb-3a5cae5b20dd,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,GIII,1619740800000,157,7,JPY,19.5333333333,bd3a462d-7f38-43ca-b96d-49cf60a9ee69,1
39888,cc4001a2-00a4-4e3a-8c55-352f1e1b808a,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,IMTB,1518048000000,109,5,EUR,58.2666666667,90fdc68e-11a8-4950-a96c-de1d3d30a636,1


In [56]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['P_ID', 'R_ID', 'TICKER'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39890


,P_ID,R_ID,TICKER,eventStrength
0,00253276-5469-43fd-84a5-859a91e90086,03961a1b-3cd6-4179-b902-7f22340d53f0,FIDU,1.0
1,00253276-5469-43fd-84a5-859a91e90086,1319fced-2e61-4d32-8779-90ee59c55ece,TMHC,1.0
2,00253276-5469-43fd-84a5-859a91e90086,2362a56e-9a20-432a-87af-4a1194038117,BME,1.0
3,00253276-5469-43fd-84a5-859a91e90086,2f5e2ea0-8185-4a94-aad7-f08eba574802,BITA,1.0
4,00253276-5469-43fd-84a5-859a91e90086,362e67d6-52fb-493a-ad11-c5a18bfa77d3,DRD,1.0
5,00253276-5469-43fd-84a5-859a91e90086,6a8395f0-97dd-4ef3-9166-6f1e22e40a24,DCOMP,1.0
6,00253276-5469-43fd-84a5-859a91e90086,6ad1d5af-3240-43e9-a1a8-c83cd848ea5e,XNET,1.0
7,00253276-5469-43fd-84a5-859a91e90086,76be5132-e354-4f97-b6f2-1a0fe50f4758,GFIN,1.0
8,00253276-5469-43fd-84a5-859a91e90086,7d20fa52-3f8d-4c02-9a24-aa6bac4a5a0f,IFGL,1.0
9,00253276-5469-43fd-84a5-859a91e90086,82ff1d93-d828-4d67-9b73-8f77ced7714a,BLCM,1.0


In [57]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['P_ID'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 31912
# interactions on Test set: 7978


In [58]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('P_ID')
interactions_train_indexed_df = interactions_train_df.set_index('P_ID')
interactions_test_indexed_df = interactions_test_df.set_index('P_ID')

In [32]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['R_ID']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [59]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('R_ID')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,R_ID,eventStrength
0,0001a0c2-cd1f-4cd2-b6eb-49234a22307a,1.0
1,aa72d402-5cf6-4969-bbfc-5f4f5897ac53,1.0
2,aa67e946-aedc-461c-8b68-77626e81873a,1.0
3,aa686373-f864-4f31-98e0-ab0089269a95,1.0
4,aa6a24d5-1263-4f58-8c04-68e34fd6bf62,1.0
5,aa6aa9cb-5163-4f24-b67f-a5d864797495,1.0
6,aa6c7805-6232-4620-8a33-c9d463ec1ce2,1.0
7,aa6d91b2-e241-4357-86cf-add74e80955e,1.0
8,aa6f886b-7468-4f3d-b909-c82d307e6083,1.0
9,aa732557-64c3-4440-b69d-4d585f0e61a5,1.0


In [60]:
# articles_df
interactions_full_df

,P_ID,R_ID,TICKER,eventStrength
0,00253276-5469-43fd-84a5-859a91e90086,03961a1b-3cd6-4179-b902-7f22340d53f0,FIDU,1.0
1,00253276-5469-43fd-84a5-859a91e90086,1319fced-2e61-4d32-8779-90ee59c55ece,TMHC,1.0
2,00253276-5469-43fd-84a5-859a91e90086,2362a56e-9a20-432a-87af-4a1194038117,BME,1.0
3,00253276-5469-43fd-84a5-859a91e90086,2f5e2ea0-8185-4a94-aad7-f08eba574802,BITA,1.0
4,00253276-5469-43fd-84a5-859a91e90086,362e67d6-52fb-493a-ad11-c5a18bfa77d3,DRD,1.0
...,...,...,...,...
39885,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,c988a10c-fe29-4ce8-a1fb-3a5cae5b20dd,GIII,1.0
39886,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,cc4001a2-00a4-4e3a-8c55-352f1e1b808a,IMTB,1.0
39887,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,d0c09c5b-3d25-4485-894a-857f25d34d3e,IPAC,1.0
39888,ffdbe0fa-5693-4bbe-a8b3-f8a1575a2201,fb5198ac-519d-4c65-857f-437b6712b11f,RBZ,1.0


In [61]:
## Content-Based Filtering
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

# item_ids = articles_df['contentId'].tolist()
item_ids = articles_df['ticker'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['name'] + "" + articles_df['exchange_iso'] + "" + articles_df['exchange_mic'] + "" + articles_df['dom_country_iso'] + "" + articles_df['dom_region'] + "" + articles_df['economic_sector'] + "" + articles_df['industry'])
tfidf_feature_names = vectorizer.get_feature_names_out()
tfidf_matrix

<495x575 sparse matrix of type '<class 'numpy.float64'>'
	with 4341 stored elements in Compressed Sparse Row format>

In [67]:
len(item_ids)

495

In [72]:
def get_item_profile(item_id):
    try:
        idx = item_ids.index(item_id)
        item_profile = tfidf_matrix[idx:idx+1]
        print(item_profile)
        return item_profile
    except:
        return None
    
def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles_list = list(filter((None).__ne__, item_profiles_list))
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
#     user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    user_item_profiles = get_item_profiles(interactions_person_df['TICKER'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
#     interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
#                                                    .isin(articles_df['contentId'])].set_index('personId')
    interactions_indexed_df = interactions_train_df[interactions_train_df['TICKER'] \
                                                   .isin(articles_df['ticker'])].set_index('P_ID')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [73]:
user_profiles = build_users_profiles()
len(user_profiles)

ValueError: blocks must be 2-D

In [75]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
#         recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
#                                     .head(topn)
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

#             recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
#                                                           left_on = 'contentId', 
#                                                           right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

In [76]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...


NameError: name 'user_profiles' is not defined